# 인덱싱·필터링·정제 실전(조회/조건/정렬/클리닝)

In [9]:
import pandas as pd

# 1) 인덱싱
데이터프레임에서 특정 행(row)과 열(column)을 선택하는 방법

분석의 정확도를 결정하는 기본기, 여기서부터 틀어질 경우 설령 통계를 제대로 하더라도 결과가 어긋날수 있다.

## 1) 필요한 행만 가져오기(Row 선택)
실제 데이터를 분석할때 대부분 "전체"를 분석하지 않는다. 보고자 하는, 얻고자 하는 데이터만을 대상으로 한다.
- 특정 날짜(어제/이번 주)만
- 특정 지점만
- 결제 완료 건만 등
이때 행 선택이 잘못되면(결제 실패 섞임, 기간 섞임 등) 매출/통계가 통째로 왜곡될 수 있다.

## 2) 필요한 열만 남기기(column 선택)
분석에 필요한 핵심 컬럼만 남겨 불필요한 컬럼을 지운다.
- 작업이 단순해짐
- 실수(컬럼 혼동)가 줄어듬
- 정제/집계/시각화가 더 빨라짐 (데이터 자체가 축소되기 떄문)

## 3) 조건에 맞는 데이터만 보기(Filtering)
분석은 보통 비교(지점별/시간대별/메뉴별)가 핵심, 비교를 하려면 먼저 조건으로 데이터를 나눠야한다.
- A점 vs B점
- 오전 vs 오후
- 평일 vs 주말
이 조건을 정확히 구해야 비교 분석이 가능해진다

# 2)  .loc vs .iloc 로 조회하기
- .loc : 라벨(이름) 기준 조회, 행·열의 이름(라벨)을 기준으로 조회
- .iloc : 위치(정수 순서) 기준 조회, 행·열의 순서(정수 인덱스)를 기준으로 조회한다.

## .loc = 라벨(이름) 기준 조회
인덱스 이름, 컬럼 이름을 기준으로 접근
- "이 이름을 가진 행/열을 가져와라"
    - 행/열 이름을 사용
    - 문자열 컬럼명 사용 가능
    - 슬라이싱 시 끝 값 포함
    - 조건 필터링과 함께 자주 사용됨
- 개수 기준, 구조 확인한다.

## .iloc = 위치(순서) 기준 조회
0, 1, 2 … 같은 순서 번호로만 접근
- "몇 번째에 있는 행/열을 가져와라"
    - 정수 인덱스만 사용
    - 파이썬 리스트 인덱싱과 동일한 규칙
    - 슬라이싱 시 끝 값 미포함
    - 구조가 확실할 때 사용하기 좋음    
- 조건 필터링, 라벨 기준 조회한다.

In [17]:
df = pd.DataFrame(
    {"menu": ["Latte", "Americano", "Mocha"], "price": [5000, 4500, 5500]},
    index=["A001", "A002", "A003"]
)
df

,menu,price
A001,Latte,5000
A002,Americano,4500
A003,Mocha,5500


In [18]:
df.loc["A002"]  # <- 라벨로 찾기

menu     Americano
price         4500
Name: A002, dtype: object

In [19]:
df.iloc[1]      # <- 위치로 찾기

menu     Americano
price         4500
Name: A002, dtype: object

## 2.1) 언제 쓰는게 자연스러운가?

In [20]:
# ------------------------------------------------------------
#    - 샘플 데이터 (카페 매출)
#    - date: 날짜(나중에 인덱스로도 써볼 예정)
#    - paid: 결제 완료 여부(True/False)
# ------------------------------------------------------------
df = pd.DataFrame({
    "date":  ["2026-01-01","2026-01-02","2026-01-03","2026-01-04","2026-01-05","2026-01-06","2026-01-07"],
    "store": ["A","A","B","A","B","A","B"],
    "menu":  ["Latte","Americano","Mocha","Latte","Americano","Mocha","Latte"],
    "qty":   [1,2,1,3,1,2,2],
    "paid":  [True, True, False, True, True, False, True]
})
df

,date,store,menu,qty,paid
0,2026-01-01,A,Latte,1,True
1,2026-01-02,A,Americano,2,True
2,2026-01-03,B,Mocha,1,False
3,2026-01-04,A,Latte,3,True
4,2026-01-05,B,Americano,1,True
5,2026-01-06,A,Mocha,2,False
6,2026-01-07,B,Latte,2,True


### .loc 이 자연스러운 상황
- 조건 필터링과 같이 쓸 때
- 인덱스/컬럼 이름이 의미가 있을 때
    - 예: 날짜가 인덱스인 경우("2026-01-01" 같은 라벨)
    - 예: df.loc["2026-01-01":"2026-01-07"]처럼 라벨 범위로 자를 때

In [21]:
# ------------------------------------------------------------
#    - 조건 필터링과 같이 쓸 때: 조건 필터링 + 필요한 컬럼만
#    - "결제 완료(True)인 행만" 가져오고
#    - 그 중에서 date/store/menu/qty 컬럼만 보고 싶을 때
#    - 패턴: df.loc[조건, [컬럼들]]
# ------------------------------------------------------------
loc_filtered = df.loc[df["paid"] == True, ["date", "store", "menu", "qty"]]
loc_filtered

,date,store,menu,qty
0,2026-01-01,A,Latte,1
1,2026-01-02,A,Americano,2
3,2026-01-04,A,Latte,3
4,2026-01-05,B,Americano,1
6,2026-01-07,B,Latte,2


In [22]:
# ------------------------------------------------------------
#    - 인덱스/컬럼 이름이 의미가 있을 때
#    - date를 인덱스로 바꾸면, 날짜 자체가 '행 이름표(라벨)'가 된다.
# ------------------------------------------------------------

df_dt = df.set_index("date")
df_dt

,store,menu,qty,paid
date,,,,
2026-01-01,A,Latte,1,True
2026-01-02,A,Americano,2,True
2026-01-03,B,Mocha,1,False
2026-01-04,A,Latte,3,True
2026-01-05,B,Americano,1,True
2026-01-06,A,Mocha,2,False
2026-01-07,B,Latte,2,True


In [23]:
# ------------------------------------------------------------
#    - 인덱스/컬럼 이름이 의미가 있을 때: 라벨(날짜) 범위로 자르기
#    - 라벨(날짜) 범위로 자르기
#    - "2026-01-01 ~ 2026-01-04" 구간만 보고 싶을 때
#    - 주의: loc은 라벨 기준이라, 범위 슬라이싱에서 끝 라벨이 포함되는 경우가 많다.
# ------------------------------------------------------------
loc_range = df_dt.loc["2026-01-01":"2026-01-04", ["store", "menu", "qty", "paid"]]
loc_range

,store,menu,qty,paid
date,,,,
2026-01-01,A,Latte,1,True
2026-01-02,A,Americano,2,True
2026-01-03,B,Mocha,1,False
2026-01-04,A,Latte,3,True


### .iloc 이 자연스러운 상황
- 위에서 몇 줄만 보기
    - 예: 첫 5행, 특정 구간(0~9행)
- 위치 기반으로 정확히 잘라야 할 때
    - 예: df.iloc[0:10] (처음 10개)
- 컬럼 이름이 헷갈릴 때 "몇 번째 열로 빠르게 확인할 때
    - 예: df.iloc[:, 0] (첫 번째 열)

In [24]:
# ------------------------------------------------------------
#    - 위에서 몇 줄만 보기 (첫 5행)
#    - iloc은 '순서(위치)' 기준: 0번째~4번째(총 5행)
#    - 주의: iloc 슬라이싱은 파이썬 규칙대로 끝(5)이 미포함
# ------------------------------------------------------------
iloc_head5 = df.iloc[0:5]
iloc_head5

,date,store,menu,qty,paid
0,2026-01-01,A,Latte,1,True
1,2026-01-02,A,Americano,2,True
2,2026-01-03,B,Mocha,1,False
3,2026-01-04,A,Latte,3,True
4,2026-01-05,B,Americano,1,True


In [25]:
# ------------------------------------------------------------
#    - 위치로 정확히 자르기
#    - 처음 3개 행만(0,1,2)
# ------------------------------------------------------------
iloc_first3 = df.iloc[0:3]
iloc_first3


,date,store,menu,qty,paid
0,2026-01-01,A,Latte,1,True
1,2026-01-02,A,Americano,2,True
2,2026-01-03,B,Mocha,1,False


In [26]:
# ------------------------------------------------------------
#    - 컬럼 이름이 헷갈릴 때 "몇 번째 열"로 빠르게 확인
#    - df.iloc[:, 0]  => 모든 행(:,)에서 첫 번째 열(0번째 열)만
# ------------------------------------------------------------
iloc_first_col = df.iloc[:, 0]
iloc_first_col

0    2026-01-01
1    2026-01-02
2    2026-01-03
3    2026-01-04
4    2026-01-05
5    2026-01-06
6    2026-01-07
Name: date, dtype: object

In [27]:
# ------------------------------------------------------------
#    - 노트북에서 print 없이 여러 결과를 한 번에 보기
#    - 셀의 마지막 줄에 (원하는 것들) 을 리스트/튜플로 묶어두면 자동 출력된다.
# ------------------------------------------------------------
df, loc_filtered, df_dt, loc_range, iloc_head5, iloc_first3, iloc_first_col

(         date store       menu  qty   paid
 0  2026-01-01     A      Latte    1   True
 1  2026-01-02     A  Americano    2   True
 2  2026-01-03     B      Mocha    1  False
 3  2026-01-04     A      Latte    3   True
 4  2026-01-05     B  Americano    1   True
 5  2026-01-06     A      Mocha    2  False
 6  2026-01-07     B      Latte    2   True,
          date store       menu  qty
 0  2026-01-01     A      Latte    1
 1  2026-01-02     A  Americano    2
 3  2026-01-04     A      Latte    3
 4  2026-01-05     B  Americano    1
 6  2026-01-07     B      Latte    2,
            store       menu  qty   paid
 date                                   
 2026-01-01     A      Latte    1   True
 2026-01-02     A  Americano    2   True
 2026-01-03     B      Mocha    1  False
 2026-01-04     A      Latte    3   True
 2026-01-05     B  Americano    1   True
 2026-01-06     A      Mocha    2  False
 2026-01-07     B      Latte    2   True,
            store       menu  qty   paid
 date        

## 2.3) 슬라이싱 핵심 정리: .loc vs .iloc (끝 포함 여부)
- .loc은 라벨 기준 슬라이싱 → 끝 값이 포함될 수 있다
- .iloc은 위치 기준 슬라이싱 → 끝 값은 항상 미포함이다
이렇게 기억하자

### 초보자용 예시(개념 이해용)
인덱스가 0,1,2,3,4일 때

- df.loc[0:2] → 0,1,2 포함 (대개 3행)
- df.iloc[0:2] → 0,1 만(2행)

### 왜 이러한 차이가 발생하는걸까?
- .loc은 라벨 구간을 자르는 개념이다.
    → “0부터 2까지”라는 의미적 범위
- .iloc은 파이썬 슬라이싱 규칙을 그대로 따른다.
    → [시작:끝] 구조, 끝은 포함하지 않음

In [6]:
df = pd.DataFrame(
    {"menu": ["Latte", "Americano", "Mocha", "Tea", "Cake"],
    "price": [5000, 4500, 5500, 4000, 6000]},
    index=[0, 1, 2, 3, 4]  # 인덱스(라벨)가 0~4인 상태
)
df

,menu,price
0,Latte,5000
1,Americano,4500
2,Mocha,5500
3,Tea,4000
4,Cake,6000


In [7]:
# .loc 슬라이싱 (라벨 기준)
df.loc[0:2]  # 0,1,2 라벨(인덱스) 포함

,menu,price
0,Latte,5000
1,Americano,4500
2,Mocha,5500


In [8]:
# .iloc 슬라이싱 (위치 기준)
df.iloc[0:2]  # 0,1 위치(인덱스)만 포함

,menu,price
0,Latte,5000
1,Americano,4500


### 가장 많이 하는 실수 (이것만 조심)

- “처음 10개만 가져오자” 할 때
    - iloc[0:10]은 정확히 10개
    - loc[0:10]은 11개가 될 수 있음

In [ ]:
df_big = pd.DataFrame({"x": range(100)}, index=range(100))
df_big

df_big.loc[0:10]    # 11개가 될 수 있음 (0~10)
df_big.iloc[0:10]   # 정확히 10개 (0~9)

first10_loc = df_big.loc[0:10]
first10_iloc = df_big.iloc[0:10]

In [33]:
first10_loc, first10_iloc

(     x
 0    0
 1    1
 2    2
 3    3
 4    4
 5    5
 6    6
 7    7
 8    8
 9    9
 10  10,
    x
 0  0
 1  1
 2  2
 3  3
 4  4
 5  5
 6  6
 7  7
 8  8
 9  9)

In [15]:
print(('loc:'),len(first10_loc))
print(('iloc:'),len(first10_iloc))


loc: 11
iloc: 10


### 기억하기 쉬운 암기 팁
- loc = label(이름표)
    - 이름으로 자른다 → 끝 포함 가능
- iloc = integer location(정수 위치)
    - 순서로 자른다 → 끝 미포함



# 3) 행/열 선택 패턴(실무에서 자주 사용됨)

pandas에서 "어떤 열을 선택했는지"에 따라 결과가 Series(1차원) 로 나오기도 하고, DataFrame(2차원) 으로 나오기도 한다.

문제는 이 둘이 지원하는 메소드/속성/출력 형태가 다르고, 그래서 다음 줄 코드가 갑자기 에러가 나거나 결과가 달라질 수 있다는 점이다.

## 3.1) 단일 컬럼 선택

### 핵심 내용
- 단일 컬럼을 선택하면 보통 Series(1차원) 가 된다
- Series와 DataFrame은 사용 가능한 속성과 메소드가 다르다

#### 예시
만약 menu 한 개 열만 보고 싶을 때:
- df["menu"] → 보통 Series
    - 특징: "한 줄짜리 컬럼"처럼 보이고, name이 menu로 붙는다.

허나
- Series는 DataFrame처럼 columns아 없다.
- 때문에 "다음 단계에서 DataFrame이라고 생각하고" 코드를 쓰면 오류가 발생한다


예시로 자주 나는 실수:
```python
- df["menu"].columns    # Series에는 columns가 없어서 오류
- df["menu"].merge(...) # merge는 DataFrame 기준이 많아서 흐름이 꼬일 수 있음
```

In [34]:
# ------------------------------------------------------------
# 예시 데이터프레임 만들기 (카페 메뉴/가격)
# ------------------------------------------------------------
df = pd.DataFrame({
    "menu":  ["Latte", "Americano", "Mocha"],
    "price": [5000, 4500, 5500],
    "paid":  [True, True, False]
})
df

,menu,price,paid
0,Latte,5000,True
1,Americano,4500,True
2,Mocha,5500,False


### 단일 컬럼 선택 → Series
pandas에서는 컬럼 하나는 “표의 한 열”이 아니라 “값의 리스트”로 취급된다.

In [64]:
menu_series = df["menu"]
menu_series

0        Latte
1    Americano
2        Mocha
Name: menu, dtype: object

### 무엇이 문제 인가?
Series를 DataFrame처럼 착각하고 다음 작업을 하면 문제가 생긴다.
```python
menu_series.columns
```
→ 오류 발생
- Series에는 columns가 없기 때문이다.

Series는 추후에 하게될
- 컬럼 추가
- 컬럼 이름 관리
- 병합 기준 테이블

작업엔 적합하지 않는다.

In [65]:
# 출력은 표처럼 보이고 컬럼 이름도 보이기 때문에 "이것도 DataFrame이겠지" 라 착각하게 되지만
# 실제로는 Series로 인식된다
menu_series

0        Latte
1    Americano
2        Mocha
Name: menu, dtype: object

In [66]:
# 컬럼이 하나이기 때문에 표(DataFrame)이 아닌 1차원 데이터(Series)로 인식
type(menu_series)

pandas.core.series.Series

In [67]:
# name 속성에 컬럼명(menu)이 들어 있음
# columns 속성은 없음
menu_series.name

'menu'

In [68]:
# 자주 하는 실수 1) Series에 .columns를 접근하려고 함 -> 오류 발생
menu_series.columns

AttributeError: 'Series' object has no attribute 'columns'

### 가져갈수 있는 결론
- 단일 컬럼 선택 = Series
- Series는 “값 처리용”
- 구조 유지가 필요하면 DataFrame으로 유지해야 한다

### (참고) 실무에서 안전한 방식: 이중 대괄호의 사용
- 컬럼을 여러 개 선택하면 결과는 DataFrame
- 단일 컬럼이라도 이중 대괄호를 쓰면 DataFrame으로 유지된다

즉, 선택 방식이 결과 타입을 결정한다 라고 볼수 있다.

In [54]:
df = pd.DataFrame({
    "menu":  ["Latte", "Americano", "Mocha"],
    "price": [5000, 4500, 5500],
    "paid":  [True, True, False]
})
df


,menu,price,paid
0,Latte,5000,True
1,Americano,4500,True
2,Mocha,5500,False


In [58]:
# 복수 컬럼 선택 → DataFrame
# 표 구조가 그대로 유지됨
# 결과 타입: DataFrame, columns 존재
menu_price_df = df[["menu", "price"]]
menu_price_df

,menu,price
0,Latte,5000
1,Americano,4500
2,Mocha,5500


In [56]:
type(menu_price_df)

pandas.core.frame.DataFrame

In [57]:
menu_price_df.columns

Index(['menu', 'price'], dtype='object')

In [63]:
# 단일 컬럼이어도 DataFrame으로 유지하는 방법 (이중대괄호의 사용)
# Series가 아닌 표 구조가 그대로 유지됨
# 컬럼이 하나여도 DataFrame
menu_df = df[["menu"]]
menu_df

,menu
0,Latte
1,Americano
2,Mocha


In [60]:
type(menu_df)

pandas.core.frame.DataFrame

In [61]:
menu_df.columns


Index(['menu'], dtype='object')

## 3.2) 복수 컬럼 선택 → DataFrame으로 남는 경우가 많다

### 핵심 내용
- 컬럼을 여러 개 선택하면 결과는 DataFrame이다
- 표 형태가 유지되기 때문에 이후 작업이 안정적이다

#### 예시
만약 여러 열을 같이 보고 싶을 때:
- df[["date","menu"]] → DataFrame
- 특징: 표 형태가 그대로 유지되고, columns가 존재한다.

### 실무에서 이게 더 안전한 이유:
"표 형태(DataFrame)"가 유지되면 이후 작업 흐름이 안정적
- 정렬(sort_values)
- 저장(to_csv)
- 결합(merge)
- 그룹화(groupby) 결과 정리
- 리포트용 표 만들기

In [77]:
# ------------------------------------------------------------
# 예시 데이터프레임 만들기 (카페 매출)
# - date, menu 등 여러 컬럼이 있는 "표" 형태
# ------------------------------------------------------------
df = pd.DataFrame({
    "date":  ["2026-01-01", "2026-01-02", "2026-01-03", "2026-01-04"],
    "store": ["A", "A", "B", "A"],
    "menu":  ["Latte", "Americano", "Mocha", "Latte"],
    "price": [5000, 4500, 5500, 5000],
    "qty":   [1, 2, 1, 3]
})

df 

,date,store,menu,price,qty
0,2026-01-01,A,Latte,5000,1
1,2026-01-02,A,Americano,4500,2
2,2026-01-03,B,Mocha,5500,1
3,2026-01-04,A,Latte,5000,3


In [78]:
# 복수 컬럼 선택하기
df_small = df[["date", "menu"]]
df_small

,date,menu
0,2026-01-01,Latte
1,2026-01-02,Americano
2,2026-01-03,Mocha
3,2026-01-04,Latte


In [79]:
type(df_small)

pandas.core.frame.DataFrame

In [80]:
df_small.columns

Index(['date', 'menu'], dtype='object')

### 왜 DataFrame으로 남기는 게 중요한가
DataFrame 상태를 유지하면 다음 작업을 형태 변경 없이 바로 할 수 있다.
- 정렬, 병렬 가능
- 결과도 다시 DataFrame으로 출력이 가능

In [76]:
sorted_df = df_small.sort_values(by="date")
sorted_df

,date,menu
0,2026-01-01,Latte
1,2026-01-02,Americano
2,2026-01-03,Mocha
3,2026-01-04,Latte


In [ ]:
menu_info = pd.DataFrame({
    "menu": ["Latte", "Americano", "Mocha"],
    "category": ["Milk", "Coffee", "Chocolate"]
})

merged_df = df_small.merge(menu_info, on="menu", how="left")
merged_df


## 3.3) “다음 코드가 달라지는” 대표 사례

### (1) 결과 모양이 달라져서 join/merge/저장이 꼬임
- Series는 저장하면 “열 이름이 애매”해질 수 있고
- DataFrame은 열 이름이 유지되어 파일/리포트에 안정적입니다.

### (2) 그룹화/집계 결과를 이어갈 때
- 단일 열로 뽑아둔 Series는 다음에 컬럼을 추가하거나 결합할 때 불편할 수 있어요.
- DataFrame은 ["colA","colB"] 형태로 확장하기가 쉽습니다.

#### “다음 코드가 달라지는” 대표 사례 정리
핵심 결론
- Series는 값 처리에 적합
- DataFrame은 표(구조) 작업에 적합

In [17]:
df = pd.DataFrame({
    "date":  ["2026-01-01","2026-01-01","2026-01-02","2026-01-02","2026-01-03"],
    "store": ["A","B","A","B","A"],
    "menu":  ["Latte","Latte","Americano","Mocha","Latte"],
    "price": [5000, 5000, 4500, 5500, 5000],
    "qty":   [1, 2, 1, 1, 3]
})
df

,date,store,menu,price,qty
0,2026-01-01,A,Latte,5000,1
1,2026-01-01,B,Latte,5000,2
2,2026-01-02,A,Americano,4500,1
3,2026-01-02,B,Mocha,5500,1
4,2026-01-03,A,Latte,5000,3


##### 사례 1. 결과 모양 차이 → 저장 / merge 흐름 차이

In [18]:
#  단일 컬럼 선택
menu_series = df["menu"]     # Series
#  단일 컬럼이라도 DataFrame으로 유지 (이중 대괄호)
menu_df     = df[["menu"]]   # DataFrame

In [19]:
type(menu_series)

pandas.core.series.Series

In [20]:
type(menu_df)

pandas.core.frame.DataFrame

왜 DataFrame이 더 안전한가

Series
- 컬럼 개념이 없음
- 저장하거나 결합할 때 추가 변환이 필요한 경우가 많음

DataFrame
- columns가 명확
- merge, 저장, 리포트 작업이 바로 이어짐

In [30]:
menu_info = pd.DataFrame({
    "menu": ["Latte", "Americano", "Mocha"],
    "category": ["Milk", "Coffee", "Chocolate"]
})

merged_df = menu_df.merge(menu_info, on="menu", how="left")
merged_df

,menu,category
0,Latte,Milk
1,Latte,Milk
2,Americano,Coffee
3,Mocha,Chocolate
4,Latte,Milk


##### 사례 2. 그룹화/집계 이후 흐름 차이

In [31]:
# 단일 집계 → Series
# 결과는 Series
# 이후 컬럼 추가, merge 전에 형태 변환이 필요해지는 경우가 많음
qty_sum_series = df.groupby("menu")["qty"].sum()
qty_sum_series


menu
Americano    1
Latte        6
Mocha        1
Name: qty, dtype: int64

In [32]:
qty_sum_df = qty_sum_series.reset_index(name="total_qty")
qty_sum_df

,menu,total_qty
0,Americano,1
1,Latte,6
2,Mocha,1


In [33]:
# 여러 집계 → DataFrame
# 처음부터 표 형태
# 컬럼 추가, 확장, 결합이 자연스럽다
qty_price_df = df.groupby("menu").agg(
    total_qty=("qty", "sum"),
    avg_price=("price", "mean")
)
qty_price_df

,total_qty,avg_price
menu,,
Americano,1,4500.0
Latte,6,5000.0
Mocha,1,5500.0


In [34]:
qty_price_df["revenue_est"] = qty_price_df["total_qty"] * qty_price_df["avg_price"]
qty_price_df

,total_qty,avg_price,revenue_est
menu,,,
Americano,1,4500.0,4500.0
Latte,6,5000.0,30000.0
Mocha,1,5500.0,5500.0


##### 내용 정리
- 단일 컬럼 선택이나 단일 집계 결과는 Series가 나오기 쉽다
- Series는 계산엔 편하지만, 구조 작업엔 불편
- 다음 단계가 있다면 DataFrame으로 유지하거나 변환하는 게 안전

한줄요약
- 다음 단계가 “표 작업”이면, 결과 타입을 DataFrame으로 맞춰라.

## 초보자에게 추천하는 실무 습관 2가지
- 표 형태가 필요하면 DataFrame으로 유지
- 항상 지금 타입(Series / DataFrame)을 의식하기


왜 이 습관이 중요한가?
- Series는 값 처리에는 편하지만 컬럼 추가, merge, 저장 같은 표 작업에는 불리하다
- DataFrame은 표 구조가 유지되어 이후 작업이 안정적이다

In [45]:
df = pd.DataFrame({
    "menu":  ["Latte", "Americano", "Mocha"],
    "price": [5000, 4500, 5500],
    "paid":  [True, True, False]
})
df


,menu,price,paid
0,Latte,5000,True
1,Americano,4500,True
2,Mocha,5500,False


### A) 표 형태(DataFrame)를 유지하고 싶으면 "이중 대괄호"를 쓰기
- pandas에서 단일 컬럼 선택은 기본적으로 Series가 된다.
- 하지만 이중 대괄호를 쓰면 단일 컬럼이어도 DataFrame으로 유지할 수 있다.


In [46]:
menu_series = df["menu"]     # Series
menu_df     = df[["menu"]]   # DataFrame

### B) 지금 내가 가진 게 Series인지 DataFrame인지 type / shape로 한 번만 확인하기
- 다음 단계에서 코드가 달라지기 때문에 한 번만 확인하는 습관을 들이면 실수를 크게 줄일 수 있다.
- type(변수): 타입 확인
- .shape    : 모양 확인
  - Series   -> (행수,)       # 열 개념이 없어서 1개 값만 나옴
  - DataFrame-> (행수, 열수)  # 표라서 2개 값이 나옴

In [43]:
type(menu_series), menu_series.shape

(pandas.core.series.Series, (3,))

In [44]:
type(menu_df), menu_df.shape

(pandas.core.frame.DataFrame, (3, 1))

# 4) 조건 필터링(불리언 인덱싱)의 본질


## 4.1) 핵심 아이디어: "True/False" 필터를 먼저 만든다
조건 필터링은 True / False로 된 필터를 먼저 만들고, 그 필터로 행을 고르는 방식이다.
- 1단계. 조건을 만족하면 True, 아니면 False 인 "필터"를 만든다.
- 2단계. 그 필터로 데이터프레임에서 True인 행만 골라낸다


### 1단계. 조건으로 True / False 필터 만들기

In [52]:
df = pd.DataFrame({
    "date":  ["2026-01-01","2026-01-01","2026-01-02","2026-01-03"],
    "menu":  ["Latte","Americano","Mocha","Latte"],
    "price": [5000, 4500, 5500, 5000],
    "qty":   [1, 2, 1, 3],
    "paid":  [True, False, True, True]
})
df

,date,menu,price,qty,paid
0,2026-01-01,Latte,5000,1,True
1,2026-01-01,Americano,4500,2,False
2,2026-01-02,Mocha,5500,1,True
3,2026-01-03,Latte,5000,3,True


In [53]:
# 만약 결제가 완료된 행만을 보고 싶다면??
mask_paid = (df["paid"] == True)
mask_paid
# 조건을 만족하면 True, 아니면 False

# 이 True / False 배열이 필터(mask)역할을 한다.

0     True
1    False
2     True
3     True
Name: paid, dtype: bool

### 2단계. 필터를 적용해서 행 고르기

In [54]:
result = df.loc[mask_paid, ["date", "menu", "price"]]
result

# 앞: 어떤 행을 고를지
# 뒤: 어떤 컬럼을 볼지
# 역할이 한 줄에서 명확히 나뉜다.

,date,menu,price
0,2026-01-01,Latte,5000
2,2026-01-02,Mocha,5500
3,2026-01-03,Latte,5000


### 왜 굳이 필터를 변수로 나눌까?
- 조건에 맞는 행이 몇 개인지 바로 확인 가능
- 여러 조건을 나눠서 디버깅 가능
- 같은 조건을 여러 분석에 재사용 가능

## 4.2) 핵심 아이디어: "True/False" 필터를 먼저 만든다
조건이 여러개로 늘어나는 경우 당연스럽게도 복잡해진다.

때문에 작성 플롯을 어느정도 정해두고 문제에 임하는게 좋다.

- 규칙 1. 조건마다 괄호 필수
- 규칙 2. and / or 대신 & / | 사용
    - and / or은 단일 True / False 비교용
    - & / |은pandas 조건 필터용

### 규칙 1. 조건마다 괄호 필수
- 괄호가 없으면 의도와 다르게 해석되거나 오류가 난다.

In [55]:
(df["paid"] == True) & (df["menu"] == "Latte")

0     True
1    False
2    False
3     True
dtype: bool

### 규칙 2. and / or 대신 & / | 사용
- and / or은 단일 True / False 비교용
- & / |은pandas 조건 필터용

In [56]:
mask = (
    (df["paid"] == True) &
    (df["menu"] == "Latte")
)

filtered = df.loc[mask, ["date", "menu", "price"]]
filtered


,date,menu,price
0,2026-01-01,Latte,5000
3,2026-01-03,Latte,5000


In [ ]:
# NOT 조건은 ~ 사용
mask_not_paid = ~(df["paid"] == True)
df.loc[mask_not_paid, ["menu", "paid"]]

## 4.3) 실무에서 가장 안전한 기본 형태: df.loc[조건, 컬럼]

조건 필터링에서 실무 기본 문장은 이 형태이다
```python
df.loc[조건, 컬럼]
```

이게 안전한 이유는 한 문장에 역할이 분명하게 나뉘기 때문이다.

- 조건 : “행을 고르는 기준”
- 컬럼 : “보여줄 열만 선택”

즉, 행 선택과 열 선택을 동시에, 명확하게 처리된다.

예시를 한번 보면
- "결제 완료인 행만" + "date, menu, price만 보여줘"
- 같은 요구를 가장 깔끔하게 표현하는 방식이 .loc이다.

### 이 문장의 역할을 먼저 이해하자
```python
df.loc[조건, 컬럼]
```
이 한 줄은 두 가지 일을 동시에 하지만, 역할이 명확히 분리되어 있다.
- 조건 : 어떤 행을 고를지
- 컬럼 : 어떤 열만 볼지

즉, 행 선택과 열 선택을 동시에, 명확하게 처리된다.

### 왜 이 형태가 실무에서 안전한가?
조건 필터링을 하다가 나오는 실수 대부분은 다음과 같다.
- 행을 고르는 코드와
- 열을 고르는 코드가
- 여러 줄로 흩어지거나 섞여 있음

df.loc[조건, 컬럼]은 “행 선택 / 열 선택”을 한 줄에 분리해서 보여준다.

그래서 코드 의도가 바로 보인다.

In [62]:
import pandas as pd

df = pd.DataFrame({
    "date":  ["2026-01-01","2026-01-01","2026-01-02","2026-01-03"],
    "menu":  ["Latte","Americano","Mocha","Latte"],
    "price": [5000, 4500, 5500, 5000],
    "qty":   [1, 2, 1, 3],
    "paid":  [True, False, True, True]
})
df

,date,menu,price,qty,paid
0,2026-01-01,Latte,5000,1,True
1,2026-01-01,Americano,4500,2,False
2,2026-01-02,Mocha,5500,1,True
3,2026-01-03,Latte,5000,3,True


In [63]:
# 1단계 조건 생성
condition = (df["paid"] == True)

In [64]:
# 2단계. 컬럼 정하기 (보여줄 열)
cols = ["date", "menu", "price"]

In [65]:
# 3단계. 기본 형태로 적용
result = df.loc[condition, cols]
result


,date,menu,price
0,2026-01-01,Latte,5000
2,2026-01-02,Mocha,5500
3,2026-01-03,Latte,5000


### 사용하는 이유?
- 조건이 복잡해져도 구조가 유지된다
- 컬럼을 바꿔도 조건 로직은 건드릴 필요 없다
- 리포트, 저장, 정렬, 그룹화로 바로 이어가기 쉽다

# 5) 정렬(sort_values, sort_index)이 필요한 이유

정렬은 분석 결과를 사람이 바로 이해할 수 있는 순서로 바꾸는 작업이다.

필터링이나 집계를 해도 정렬이 없으면 “뭐가 중요한지”가 바로 보이지 않는다.

## 5.1) 실무에서 가장 많이 쓰는 정렬 패턴
리포트에서 가장 자주 나오는 요구는 아래 세 가지다.
- 최신순: 최근 데이터부터 보기
- 큰 값 순: 매출 / 수량 / 점수 높은 순
- TOP N: 상위 몇 개만 보여주기

이건 거의 전부 sort_values로 해결한다.

In [87]:
df = pd.DataFrame({
    "date": ["2026-01-03", "2026-01-01", "2026-01-02"],
    "menu": ["Latte", "Americano", "Mocha"],
    "qty":  [2, 1, 3],
    "price":[5000, 4500, 5500]
})

df["revenue"] = df["qty"] * df["price"]
df


,date,menu,qty,price,revenue
0,2026-01-03,Latte,2,5000,10000
1,2026-01-01,Americano,1,4500,4500
2,2026-01-02,Mocha,3,5500,16500


In [88]:
# 1) 최신순 정렬 (날짜 기준 내림차순)
latest = df.sort_values(by="date", ascending=False)
latest

,date,menu,qty,price,revenue
0,2026-01-03,Latte,2,5000,10000
2,2026-01-02,Mocha,3,5500,16500
1,2026-01-01,Americano,1,4500,4500


In [89]:
# 2) 매출 큰 순 정렬
revenue_rank = df.sort_values(by="revenue", ascending=False)
revenue_rank

,date,menu,qty,price,revenue
2,2026-01-02,Mocha,3,5500,16500
0,2026-01-03,Latte,2,5000,10000
1,2026-01-01,Americano,1,4500,4500


In [90]:
# 3) TOP N 만들기 (상위 2개)
top2 = revenue_rank.head(2)
top2

,date,menu,qty,price,revenue
2,2026-01-02,Mocha,3,5500,16500
0,2026-01-03,Latte,2,5000,10000


## 5.2) 정렬을 하면 뭐가 달라지는가
- 정렬 전
    - 어떤 메뉴가 1등인지 바로 안 보임
- 정렬 후
    - 상위 / 하위가 바로 보임
    - 우선순위가 명확해짐

때문에 정렬은 의사결정을 위한 마지막 정리 단계라고 볼 수 있다.

In [91]:
# 샘플: 카페 주문 데이터
df = pd.DataFrame({
    "menu": ["Latte","Americano","Mocha","Latte","Mocha","Americano","Tea","Tea","Latte"],
    "qty":  [2, 1, 1, 3, 2, 4, 5, 1, 1],
    "price":[5000,4500,5500,5000,5500,4500,4000,4000,5000]
})

# 매출(=수량*가격) 컬럼 추가
df["revenue"] = df["qty"] * df["price"]

df

,menu,qty,price,revenue
0,Latte,2,5000,10000
1,Americano,1,4500,4500
2,Mocha,1,5500,5500
3,Latte,3,5000,15000
4,Mocha,2,5500,11000
5,Americano,4,4500,18000
6,Tea,5,4000,20000
7,Tea,1,4000,4000
8,Latte,1,5000,5000


In [92]:
# 메뉴별 매출 집계표 만들기
menu_sales = df.groupby("menu", as_index=False)["revenue"].sum()
menu_sales

,menu,revenue
0,Americano,22500
1,Latte,30000
2,Mocha,16500
3,Tea,24000


In [93]:
# 1) 정렬 안 한 집계표: 순서가 애매해서 TOP 메뉴가 바로 안 보일 수 있음
menu_sales_unsorted = menu_sales
menu_sales_unsorted

,menu,revenue
0,Americano,22500
1,Latte,30000
2,Mocha,16500
3,Tea,24000


In [94]:
# 2) 매출 내림차순 정렬: TOP 메뉴가 즉시 보임 (TOP 3)
menu_sales_sorted = menu_sales.sort_values(by="revenue", ascending=False)
top5 = menu_sales_sorted.head(3)
top5

,menu,revenue
1,Latte,30000
3,Tea,24000
0,Americano,22500


In [95]:
# 3) 하위 메뉴(개선 대상)도 바로 보임 (BOTTOM 3)
bottom3 = menu_sales_sorted.tail(3)
bottom3

,menu,revenue
3,Tea,24000
0,Americano,22500
2,Mocha,16500


## 5.3) sort_values vs sort_index (감 잡기)

정렬은 분석 결과를 “읽히는 결과”로 바꾸는 마지막 기본 단계, 리포트에서 요구하는 최신순/Top N/우선순위 표는 대부분 정렬로 완성된다.

#### sort_values
- 값 기준 정렬
- 매출, 수량, 점수, 가격 등
- 리포트 / 랭킹표에서 가장 많이 사용

In [96]:
# 매출 큰 순 정렬
df.sort_values(by="revenue", ascending=False)

,menu,qty,price,revenue
6,Tea,5,4000,20000
5,Americano,4,4500,18000
3,Latte,3,5000,15000
4,Mocha,2,5500,11000
0,Latte,2,5000,10000
2,Mocha,1,5500,5500
8,Latte,1,5000,5000
1,Americano,1,4500,4500
7,Tea,1,4000,4000


#### sort_index
- 인덱스(행 이름표) 기준 정렬
- 날짜가 인덱스일 때
- groupby 결과를 정리할 때

In [97]:
# 날짜 인덱스 정렬
daily = pd.DataFrame({
    "revenue": [12000, 8000, 15000]
}, index=["2026-01-03", "2026-01-01", "2026-01-02"])

daily.sort_index()


,revenue
2026-01-01,8000
2026-01-02,15000
2026-01-03,12000


In [ ]:
# ------------------------------------------------------------
# sort_values vs sort_index 차이 한 번에 감 잡기
# - sort_values: "값" 기준 정렬 (리포트/랭킹표에서 가장 자주)
# - sort_index : "인덱스(행 이름표)" 기준 정렬 (시간축/인덱스 기반 표 정리)
# ------------------------------------------------------------

# 1) sort_values 예시: 메뉴별 매출(값)로 정렬
sales = pd.DataFrame({
    "menu": ["Latte", "Americano", "Mocha", "Tea"],
    "revenue": [32000, 15000, 27000, 9000]
})

# 값(revenue) 기준 내림차순 정렬 -> "매출 TOP" 랭킹표 만들 때
sales_sorted_by_value = sales.sort_values(by="revenue", ascending=False)


# 2) sort_index 예시: 날짜를 인덱스로 둔 뒤 인덱스(날짜 라벨)로 정렬
daily = pd.DataFrame({
    "date": ["2026-01-03", "2026-01-01", "2026-01-02"],
    "revenue": [12000, 8000, 15000]
}).set_index("date")   # date가 인덱스(행 이름표)가 됨

# 인덱스(날짜) 기준 오름차순 정렬 -> 시간 흐름대로 정리할 때
daily_sorted_by_index = daily.sort_index(ascending=True)


# 3) sort_index 예시(그룹화 결과 정리): groupby 결과는 인덱스가 menu가 되는 경우가 많음
orders = pd.DataFrame({
    "menu": ["Latte","Americano","Latte","Mocha","Mocha","Tea"],
    "qty":  [2, 1, 3, 1, 2, 4]
})

qty_sum = orders.groupby("menu")["qty"].sum()  # 결과: 인덱스가 menu인 Series
qty_sum_sorted_by_index = qty_sum.sort_index() # 인덱스(메뉴 이름) 알파벳/가나다 순 정리


sales, sales_sorted_by_value, daily, daily_sorted_by_index, qty_sum, qty_sum_sorted_by_index

# 6) 클리닝(정제)의 4대 문제
클리닝은 분석이 가능하도록 데이터를 사람 기준으로 정리하는 과정 이다.

실무에서 거의 항상 마주치는 문제는 4가지다.
- 컬럼명/구조 문제:     rename, drop
- 문자열 문제:          공백, 대소문자, 불필요 문자(“원”, “,”)
- 결측치 문제:          NaN(비어 있음)
- 중복 문제:            같은 행이 여러 번 있음


클리닝은 “지저분한 데이터를 분석 가능한 표로 바꾸는 작업”이며,

컬럼 → 문자열 → 결측치 → 중복 순서로 처리하면 흐름이 꼬이지 않는다.

In [100]:
# 일부러 문제를 섞어 만든 원본 데이터
raw = pd.DataFrame({
    " Date ": ["2026-01-01", "2026-01-01", "2026-01-02", None,        "2026-01-02"],
    " Menu ": [" Latte ",   " latte",      "Americano ", "Mocha",     "Americano "],
    " Price ": ["5,000원",   "5,000원",     None,         "5,500원",   None],
    " Qty ":   [1,           1,            2,            None,        2],
    " memo ":  ["test",      "test",       "dup",        "x",         "dup"]  # 분석에 필요 없는 컬럼이라고 가정
})

raw

,Date,Menu,Price,Qty,memo
0,2026-01-01,Latte,"5,000원",1.0,test
1,2026-01-01,latte,"5,000원",1.0,test
2,2026-01-02,Americano,None,2.0,dup
3,None,Mocha,"5,500원",NaN,x
4,2026-01-02,Americano,None,2.0,dup


## 6.1) 컬럼명/구조 문제: rename, drop
### 발생한 문제
- 컬럼명에 공백, 대소문자 섞임
- 분석에 필요 없는 컬럼 존재

### 해결법
 - 컬럼명 정리
 - 불필요한 컬럼 제거

In [104]:
df = raw.copy()

df.columns = df.columns.str.strip().str.lower()   # " Date " -> "date"
df = df.rename(columns={"menu": "menu_name"})     # 예: menu -> menu_name
df = df.drop(columns=["memo"])                   # 필요 없는 컬럼 제거

df

,date,menu_name,price,qty
0,2026-01-01,Latte,"5,000원",1.0
1,2026-01-01,latte,"5,000원",1.0
2,2026-01-02,Americano,None,2.0
3,None,Mocha,"5,500원",NaN
4,2026-01-02,Americano,None,2.0


## 6.2) 문자열 문제: 공백/대소문자/불필요 문자 제거
### 발생한 문제
- 공백, 대소문자 혼용
- 숫자인데 문자열로 되어 있음 ("5,000원")

### 해결법
- 문자열 정리 후 숫자로 변환

In [105]:
# 메뉴 이름 정리
df["menu_name"] = df["menu_name"].str.strip().str.lower()

# 가격 정리: "5,000원" -> 5000
df["price"] = (
    df["price"]
    .astype("string")
    .str.replace(",", "", regex=False)
    .str.replace("원", "", regex=False)
)

# 숫자로 변환 (변환 불가한 값은 NaN)
df["price"] = pd.to_numeric(df["price"], errors="coerce")

df


,date,menu_name,price,qty
0,2026-01-01,latte,5000,1.0
1,2026-01-01,latte,5000,1.0
2,2026-01-02,americano,<NA>,2.0
3,None,mocha,5500,NaN
4,2026-01-02,americano,<NA>,2.0


## 6.3) 결측치(NaN) 문제: dropna vs fillna

### 선택 가능한 2가지 전략
- A: 버린다 (dropna)
- B: 채운다 (fillna)

### A: 버린다 (dropna)
- 데이터 신뢰도가 중요할 때
- 단순하고 빠름

In [106]:
df_drop = df.dropna(subset=["price", "qty"])
df_drop

,date,menu_name,price,qty
0,2026-01-01,latte,5000,1.0
1,2026-01-01,latte,5000,1.0


### B: 채운다 (fillna)
- 데이터를 최대한 살리고 싶을 때
- 실무에서 더 자주 사용

In [107]:
df_fill = df.copy()

# 수량: 비어 있으면 1로 가정
df_fill["qty"] = df_fill["qty"].fillna(1)

# 가격: 메뉴별 평균 가격으로 채우기
menu_mean_price = df_fill.groupby("menu_name")["price"].transform("mean")
df_fill["price"] = df_fill["price"].fillna(menu_mean_price)

df_fill

,date,menu_name,price,qty
0,2026-01-01,latte,5000,1.0
1,2026-01-01,latte,5000,1.0
2,2026-01-02,americano,<NA>,2.0
3,None,mocha,5500,1.0
4,2026-01-02,americano,<NA>,2.0


## 6.4) 중복 문제: duplicated / drop_duplicates
### 발생한 문제
- 같은 주문이 여러 번 들어 있음

### 해결법
- 중복 기준을 명확히 정한다
- 첫 번째 / 마지막 중 무엇을 남길지 결정한다

- subset: 무엇을 기준으로 중복인지 판단할지
- keep: "first" (첫 행 유지) or "last" (마지막 행 유지)

In [108]:
dedup = df_fill.drop_duplicates(
    subset=["date", "menu_name", "qty", "price"],
    keep="first"
)

dedup


,date,menu_name,price,qty
0,2026-01-01,latte,5000,1.0
2,2026-01-02,americano,<NA>,2.0
3,None,mocha,5500,1.0


In [109]:
# 중복 기준 예시: date + menu_name + qty + price 가 같으면 같은 주문으로 보자
dup_mask = df_fill.duplicated(subset=["date", "menu_name", "qty", "price"], keep="first")
dup_mask

0    False
1     True
2    False
3    False
4     True
dtype: bool

In [110]:
# 중복 제거(첫 번째 유지)
dedup_first = df_fill.drop_duplicates(subset=["date", "menu_name", "qty", "price"], keep="first")
dedup_first

,date,menu_name,price,qty
0,2026-01-01,latte,5000,1.0
2,2026-01-02,americano,<NA>,2.0
3,None,mocha,5500,1.0


In [111]:
# 중복 제거(마지막 유지)
dedup_last = df_fill.drop_duplicates(subset=["date", "menu_name", "qty", "price"], keep="last")
dedup_last

,date,menu_name,price,qty
1,2026-01-01,latte,5000,1.0
3,None,mocha,5500,1.0
4,2026-01-02,americano,<NA>,2.0
